In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import tensorflow as tf
import heartpy as hp
import pandas as pd
import numpy as np 
import pywt as pw
import openpyxl
import pickle
import tsfel
import json

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Functions 

In [ ]:
num_of_participants = 23
num_of_clips = 18

#In Hz:
cutoff = [0.5, 45]
sample_rate = 256.0
order = 3
filtertype ='bandpass'
def data_filter (data):
    # Separate data by channel
    for participant in data.keys():
        #print("Starting " + participant)
        for clip_num in range(0, num_of_clips):
            # Filter both channels from baseline
            CH1 = [row[0] for row in data[participant]['ECG']['baseline'][clip_num]]
            CH1_filtered = hp.filter_signal(CH1, cutoff = cutoff, sample_rate = sample_rate, order = order, filtertype=filtertype)
            # CH1_filtered = hp.butter_lowpass_filter(data, cutoff=cutoff, sample_rate=sample_rate, order=order)

            CH2 = [row[1] for row in data[participant]['ECG']['baseline'][clip_num]]
            CH2_filtered = hp.filter_signal(CH2, cutoff = cutoff, sample_rate = sample_rate, order = order, filtertype=filtertype)
            # CH2_filtered = hp.butter_lowpass_filter(data, cutoff=cutoff, sample_rate=sample_rate, order=order)

            s_clip = "clip " + str(clip_num+1) 

            data[participant]['ECG'][s_clip] = {}
            data[participant]['ECG'][s_clip]['CH1'] = {}
            data[participant]['ECG'][s_clip]['CH1']['baseline'] = CH1_filtered.tolist()
            data[participant]['ECG'][s_clip]['CH2'] = {}
            data[participant]['ECG'][s_clip]['CH2']['baseline'] = CH2_filtered.tolist()

            
            # Filter both channels from stimuli
            CH1 = [row[0] for row in data[participant]['ECG']['stimuli'][clip_num]]
            CH1_filtered = hp.filter_signal(CH1, cutoff = cutoff, sample_rate = sample_rate, order = order, filtertype=filtertype)
            # CH1_filtered = hp.butter_lowpass_filter(data, cutoff=cutoff, sample_rate=sample_rate, order=order)

            CH2 = [row[1] for row in data[participant]['ECG']['stimuli'][clip_num]]
            CH2_filtered = hp.filter_signal(CH2, cutoff = cutoff, sample_rate = sample_rate, order = order, filtertype=filtertype)
            # CH2_filtered = hp.butter_lowpass_filter(data, cutoff=cutoff, sample_rate=sample_rate, order=order)

            data[participant]['ECG'][s_clip]['CH1']['stimuli'] = CH1_filtered.tolist()
            data[participant]['ECG'][s_clip]['CH2']['stimuli'] = CH2_filtered.tolist()

            data[participant]['ECG'][s_clip]['ScoreValence'] = data[participant]['ScoreValence'][clip_num]
            data[participant]['ECG'][s_clip]['ScoreArousal'] = data[participant]['ScoreArousal'][clip_num]
            data[participant]['ECG'][s_clip]['ScoreDominance'] = data[participant]['ScoreDominance'][clip_num]
            
            print("Done filtering raw for " + participant + " at clip#" + str(clip_num+1) + "                 ", end="\r", flush=True)
        data[participant]['ECG'].pop('stimuli')
        data[participant]['ECG'].pop('baseline')

        data[participant].pop('ScoreValence')
        data[participant].pop('ScoreArousal')
        data[participant].pop('ScoreDominance')
    
    print("All done.... Highpass filter applied, cutoff: [" + str(cutoff[0]) + "," + str(cutoff[1]) + "]Hz")
    return data
            
            
### Function to display graph
def show_graph(participant, clip, dtype, range_min = 0, range_max = -1):
    plt.plot(filtered_data[participant]['ECG'][dtype][clip][range_min:range_max])
    plt.ylabel("mV")
    plt.xlabel("Sample #")
    graph_title = participant + "  clip#" + str(clip) + " " + dtype + " data range: " + str(range_min) + " => " + str(range_max)
    plt.title(graph_title)

### Function to split channels
def chan_split(filtered_data, participant, clip, dtype):
    ch1 = [row[0] for row in filtered_data[participant]['ECG'][dtype][clip]]
    ch2 = [row[1] for row in filtered_data[participant]['ECG'][dtype][clip]]
    
    return ch1, ch2


### Function to plot Heard Rate Signal Peak Detection
def peak_detection(participant, clip, dtype, ch = 1, range_min = 0, range_max = -1):
    
    p1_c1_ch1, p1_c1_ch2 =(chan_split(participant, clip, dtype))
    wd1 = hp.process(p1_c1_ch1[range_min:range_max], 256)
    wd2 = hp.process(p1_c1_ch1[range_min:range_max], 256)
    if ch == 1:
        hp.plotter(wd1[0], wd1[1])
    else:
        hp.plotter(wd2[0], wd2[1])

def extract_time_domain_as_pd_df(processed_data):
    combined_data = pd.DataFrame()
    cfg = tsfel.get_features_by_domain()
    for CH in ['CH1', 'CH2']:
        for participant in processed_data:
            for clip in processed_data[participant]['ECG']:
                data_size = len(processed_data[participant]['ECG'][clip][CH]['stimuli'])
                for_iterations = windows_size % data_size
                for i in range(0, data_size, windows_size):
                    print("Processing " + participant + " " + clip + " " + CH + " window:" + str(int(i/windows_size)) + "/" + str(int(data_size/windows_size)) + "          ", end="\r", flush=True)
                    features_df = pd.DataFrame()
                    features_df = tsfel.time_series_features_extractor(cfg, processed_data[participant]['ECG'][clip][CH]['stimuli'][i:i+windows_size], verbose=False)
                    features_df['ScoreValence'] = data[participant]['ECG'][clip]['ScoreValence']
                    features_df['ScoreArousal'] = data[participant]['ECG'][clip]['ScoreArousal']
                    features_df['ScoreDominance'] = data[participant]['ECG'][clip]['ScoreDominance']
                
                    combined_data = pd.concat([combined_data, features_df], ignore_index=True)
                
                    # print("Done with time features for participant ", participant, " clip ", clip)
    print()
    print("Done extracting time domain features")
    return combined_data

def extract_ecg_features(processed_data):
    combined_data = pd.DataFrame()
    for CH in ['CH1', 'CH2']:
        for participant in processed_data:
            for clip in processed_data[participant]['ECG']:
                print("Processing " + participant + " " + clip + " " + CH + "          ", end="\r", flush=True)
                features_df = pd.DataFrame()
                hp_tupple = hp.process(processed_data[participant]['ECG'][clip][CH]['stimuli'], 256)
                features_df = pd.DataFrame([hp_tupple[1]])
                features_df['ScoreValence'] = processed_data[participant]['ECG'][clip]['ScoreValence']
                features_df['ScoreArousal'] = processed_data[participant]['ECG'][clip]['ScoreArousal']
                features_df['ScoreDominance'] = processed_data[participant]['ECG'][clip]['ScoreDominance']
            
                combined_data = pd.concat([combined_data, features_df], ignore_index=True)
            
                # print("Done with time features for participant ", participant, " clip ", clip)
    print()
    print("Done extracting time domain features")
    return combined_data
            

# Code

## Load & Process RAW Data, then create and save processed data to .pkl

In [ ]:
raw_data = {}
load_raw_from = '../data_set_raw_ECG/raw_data_dict.pkl'

with open(load_raw_from, 'rb') as f:
    raw_data = pickle.load(f)

filtered_data = data_filter(raw_data)

plt.plot(filtered_data['participant_1']['ECG']['clip 1']['CH1']['stimuli'][0:1024])

In [ ]:
basic_features = extract_ecg_features(filtered_data)

In [ ]:
basic_features

In [ ]:
X = basic_features.drop(['ScoreValence', 'ScoreArousal', 'ScoreDominance'], axis=1)
Y = basic_features[['ScoreValence', 'ScoreArousal', 'ScoreDominance']]  # Ensure correct DataFrame is used

# Normalize data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X_normalized, Y, test_size=0.2, random_state=42)

In [ ]:
# Train the SVM
svm_rbf_model = SVC(kernel='rbf', C=100, gamma=1)  # Correctly named SVM model
multi_target_svm = MultiOutputClassifier(svm_rbf_model)
multi_target_svm.fit(X_train, Y_train)

# Predict and evaluate
Y_pred = multi_target_svm.predict(X_test)

# Evaluate each target
for i, target in enumerate(['Valence', 'Arousal', 'Dominance']):
    accuracy = accuracy_score(Y_test.iloc[:, i], Y_pred[:, i])
    print(f"Accuracy for {target}:", accuracy)

# THIS IS A GOOD OPTION TOO ^^^

In [ ]:
# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Multi-Output SVM Classifier
# Best parameters: {'estimator__C': 100, 'estimator__gamma': 1, 'estimator__kernel': 'rbf'}
svm_rbf_model = SVC(kernel='rbf', C=100, gamma=1)  # Base SVM model
multi_target_svm = MultiOutputClassifier(svm_rbf_model)  # Multi-output wrapper

# Training
multi_target_svm.fit(X_train_scaled, Y_train)

# Predictions
y_pred = multi_target_svm.predict(X_test_scaled)

# Evaluation
for i, target in enumerate(['Valence', 'Arousal', 'Dominance']):
    print(f"Classification Report for {target}:")
    print(classification_report(Y_test.iloc[:, i], y_pred[:, i]))

# Grid Search (Note: This is tricky with multi-output classifiers)
# For simplicity, the following code is a template for a single-output grid search
param_grid = {
    'estimator__C': [0.1, 1, 10, 100],  # Note the 'estimator__' prefix
    'estimator__gamma': [1, 0.1, 0.01, 0.001],
    'estimator__kernel': ['rbf', 'linear']
}
grid_search = GridSearchCV(MultiOutputClassifier(SVC()), param_grid, refit=True, verbose=2)
grid_search.fit(X_train_scaled, Y_train)

# Best parameters
print("Best parameters:", grid_search.best_params_)